In [1]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train = pd.read_pickle("clean-data/train.pkl")
train.head()

,order_id,user_id,platform,personal_or_business,placement_date,placement_weekday,placement_time,confirm_date,confirm_weekday,confirm_time,...,distance,temperature,pickup_lat,pickup_long,dest_lat,dest_long,rider_id,pickup_to_arrival_time,placement_to_confirm_time,confirm_to_pickup_time
0,4211,633,3,Business,9,5,34546,9,5,34810,...,4,20.400000,-1.317755,36.830370,-1.300406,36.829741,432,745,264,2840
1,25375,2285,3,Personal,12,5,40576,12,5,41001,...,16,26.400000,-1.351453,36.899315,-1.295004,36.814358,856,1993,425,1248
2,1899,265,3,Business,30,2,45565,30,2,45764,...,3,23.258889,-1.308284,36.843419,-1.300921,36.828195,155,455,199,619
3,9336,1402,3,Business,15,5,33934,15,5,33965,...,9,19.200000,-1.281301,36.832396,-1.257147,36.795063,855,1341,31,1021
4,27883,1737,1,Personal,13,1,35718,13,1,35778,...,9,15.400000,-1.266597,36.792118,-1.295041,36.809817,770,1214,60,545


In [3]:
train_pca = pd.read_pickle("clean-data/train-pca.pkl")
train_pca.head()

,PC1,PC2,PC3,PC4,PC5,platform1,platform2,platform3,personal,pickup_to_arrival_time
0,0.473153,0.280127,-0.237710,0.594922,-0.734770,0,0,1,0,745
1,0.620531,1.506519,-0.507940,-0.517865,0.413051,0,0,1,1,2886
2,0.532004,-0.493915,0.798359,0.154547,0.111239,0,0,1,0,2615
3,0.629128,0.786396,-2.134154,1.070596,-1.274667,0,0,1,1,2986
4,0.547350,-0.717573,-0.642367,0.497750,-0.989003,0,1,0,1,1602


In [6]:
from sklearn.linear_model import LinearRegression
pca_model = LinearRegression()
X = train_pca.iloc[:,:-1]
pca_model.fit(X=X, y=train_pca["pickup_to_arrival_time"])

LinearRegression()